# Assignment 3

> Joshua Schmidt (jns223)

In [4]:
from os import listdir
from os.path import isfile, join
from os import system
import pandas as pd
import numpy as np
from glob import glob

import gzip
import librosa
import librosa.display

try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO

import matplotlib.pyplot as plt

import folium

# make figures easier to annotate
import mplcursors

import geopy
from geopy.geocoders import Nominatim
import plotly.graph_objects as go

from IPython.display import Audio

# Loading 

In [5]:
# Here are the sample code to load the three types of data. Change it based on your needs. 

location_dir = "./data/person2-1DBB0F6F-1F81-4A50-9DF4-CD62ACFA4842.absolute_locations.csv.gz"
location = pd.read_csv(location_dir, compression='gzip')

feature_label_dir = "./data/person2-1DBB0F6F-1F81-4A50-9DF4-CD62ACFA4842.features_labels.csv.gz"
feature_label = pd.read_csv(feature_label_dir, compression='gzip')

#read in one audio-MFCC sample to 2d numpy array. Make it into a function if you need. 
timestamp = 1440652787
mfcc_extension = ".sound.mfcc"
mfcc = "./data/person2-MFCC-1DBB0F6F-1F81-4A50-9DF4-CD62ACFA4842/"

df = pd.read_csv(mfcc + str(timestamp) + mfcc_extension, sep=",", header=None, )
df = df.drop(columns=[13])
mfcc_coeff = df.T.to_numpy()



### Here are what are in the loaded data

In [6]:
location

,timestamp,latitude,longitude
0,1440627472,32.882277,-117.234632
1,1440627533,32.882289,-117.234622
2,1440627593,32.882289,-117.234629
3,1440627654,32.882292,-117.234630
4,1440627712,32.882284,-117.234628
...,...,...,...
7370,1441292839,32.878956,-117.231589
7371,1441292931,32.878955,-117.231589
7372,1441292959,32.878956,-117.231589
7373,1441293052,32.878956,-117.231589


In [7]:
feature_label

,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,...,label:STAIRS_-_GOING_DOWN,label:ELEVATOR,label:OR_standing,label:AT_SCHOOL,label:PHONE_IN_HAND,label:PHONE_IN_BAG,label:PHONE_ON_TABLE,label:WITH_CO-WORKERS,label:WITH_FRIENDS,label_source
0,1440627472,0.978119,0.002341,0.002273,0.005612,0.976953,0.978090,0.979302,1.296142,6.684609,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,4
1,1440627533,0.978315,0.001636,0.000510,0.002482,0.977402,0.978380,0.979202,2.174581,6.684610,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,4
2,1440627593,0.978582,0.002582,-0.002260,0.004173,0.977297,0.978632,0.980001,2.162259,6.684608,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,4
3,1440627654,0.978640,0.001695,-0.000799,0.002433,0.977607,0.978664,0.979654,2.282570,6.684610,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,4
4,1440627712,0.978938,0.001981,0.001092,0.003347,0.977863,0.978961,0.979923,1.968584,6.684610,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7370,1441292839,1.014379,0.001043,0.000587,0.001415,1.013690,1.014335,1.015096,2.438258,6.684611,...,NaN,NaN,0.0,0.0,0.0,NaN,1.0,NaN,NaN,1
7371,1441292931,1.013954,0.001003,0.000501,0.001301,1.013225,1.013953,1.014676,2.625195,6.684611,...,NaN,NaN,0.0,0.0,0.0,NaN,1.0,NaN,NaN,1
7372,1441292959,1.014174,0.000964,-0.000307,0.001276,1.013541,1.014194,1.014796,2.497071,6.684611,...,NaN,NaN,0.0,0.0,0.0,NaN,1.0,NaN,NaN,1
7373,1441293052,1.014009,0.001029,-0.000293,0.001391,1.013350,1.013978,1.014709,2.467187,6.684611,...,NaN,NaN,0.0,0.0,0.0,NaN,1.0,NaN,NaN,1


In [9]:
mfcc_coeff 

array([[-8.24621125e+01, -9.93218702e+00, -4.97305810e+00, ...,
         2.14531050e+00,  2.66927492e+00,  1.51974319e+00],
       [ 8.61659633e-16, -1.19795986e+00, -1.28985223e+00, ...,
        -5.51409700e-01, -9.40378632e-01, -6.41590596e-01],
       [-4.30829816e-16,  5.51628363e-01,  5.65086712e-01, ...,
         6.14282306e-01,  5.58024678e-01,  8.11163369e-01],
       ...,
       [ 5.16995780e-15, -2.67070573e-01, -2.49183667e-01, ...,
        -1.69268257e-01, -3.08869254e-01, -2.23282110e-01],
       [ 8.18576651e-15, -1.13510040e-01, -1.27752455e-01, ...,
        -1.50372953e-01, -1.58451437e-01, -1.95369635e-01],
       [-1.16324050e-14,  2.10808326e-01,  1.17075395e-01, ...,
        -3.44395678e-04,  3.65590400e-02, -4.13183255e-02]])

In [8]:
mfcc_coeff.shape

(13, 332)

In [ ]:
fig = go.Figure(go.Densitymapbox(lat=sleeping_df.latitude, lon=sleeping_df.longitude,
                                 radius=5))
fig.update_layout(mapbox_style="open-street-map", mapbox_center_lon=avg_lon)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, title = "Eating ")
fig.show()
